# **Chatbot**

We're here to help answer all your questions and guide you on your learning journey.

Our chatbot is equipped with a vast knowledge base and is designed to provide accurate and reliable information. We understand that every student has unique needs, so feel free to ask any questions, no matter how big or small. We're committed to supporting your academic growth and ensuring you have a seamless learning experience.

In [ ]:
# Download the necessary resources from the Natural Language Toolkit (NLTK) library
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### **Importing Neccessary libraries and modules**

#### Intel optimizations
`from tensorflow import keras` Imports the Keras module from TensorFlow, a high-level neural networks API  
`from tensorflow.keras import mixed_precision` Imports the mixed-precision module from Keras for mixed-precision training.  
`policy = mixed_precision.Policy('mixed_bfloat16')` Sets the mixed-precision policy to use mixed bfloat16 data type.   
`mixed_precision.set_global_policy(policy)` Applies the mixed-precision policy globally.

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
from tensorflow import keras
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_bfloat16')
mixed_precision.set_global_policy(policy)
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [ ]:
# The variables 'words', 'classes', 'documents', and 'ignore_words' are essential components used during the training process of the chatbot.
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

# Open the 'intents.json' file containing the predefined intents and their corresponding data.
data_file = open('intents.json').read()

#Read the contents of the file and load it as a JSON object.
intents = json.loads(data_file)


### **Tokenizing Patterns and Creating Corpus**

We are working with a list of intents, where each intent contains a set of patterns. The purpose is to tokenize each word within the patterns and create a corpus for further processing.

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


### **Create Training Data**

This code takes the patterns and intent tags, tokenizes and lemmatizes the words, and constructs a bag-of-words representation for each pattern.

It also prepares the corresponding intent labels in a one-hot encoded format. The resulting training data is shuffled and organized into train_x (patterns) and train_y (intent labels) lists, which are ready for training a model for intent classification.

In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in an attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in the current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for the current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

### **Create Model**
This code builds and trains a neural network model for a chatbot, using a three-layer architecture with varying numbers of neurons. The model is compiled with an appropriate optimizer and loss function, and the training process is performed for a specified number of epochs. The resulting model is then saved for future use.

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("Model created")

### **Necessary imports and loading of files for the chatbot model.**

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

## **Functions Definitions**



### **Cleaning up the Sentence**

Cleaning up user input before feeding it into the chatbot model. It tokenizes the sentence and applies lemmatization and lowercase conversion to improve matching and understanding during the intent classification and response generation processes.

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

### **Bag Of Words Function**

Converting a sentence into a bag of words representation using the provided vocabulary. The resulting bag of words representation can be used as input for the chatbot model to predict the intent or generate appropriate responses based on the user query.

In [ ]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))


### **Predicting Intent Class**

Enabling the chatbot to predict the intent class for a given input sentence, allowing it to determine the appropriate response based on the predicted intent.

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

### **Retrieving Response Based on Intents**
Fetching an appropriate response based on the predicted intent, providing dynamic and contextually relevant answers to user queries.

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


### **Chatbot Response Function**
Function serves as the core component of the chatbot's conversational logic.

 It takes user queries as input, predicts the intent class, and retrieves the corresponding response. The function acts as a bridge between the user's input and the trained model, allowing the chatbot to provide relevant and context-specific responses.

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res


## **User-Chatbot Interaction Loop**
 The user can enter their input, and the chatbot will generate a response based on the provided input.

 The conversation continues until the user explicitly enters 'exit', allowing for an interactive and dynamic chatbot experience.

In [ ]:
user_input = input("You: ")
while user_input.lower() != 'exit':
    response = chatbot_response(user_input)
    print("Bot:", response)
    user_input = input("You: ")
